# **数据处理 （EDA）**

这里我们选取著名的泰坦尼克数据作为例子，学习数据的处理的各个步骤。

# **数据读取**

首先加载将要使用的软件包。

In [ ]:
import pandas as pd
import numpy as np
import os
# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

泰坦尼克的数据已经上载到当前子目录。下面的命令显示当前目录'/kaggle'下的文件。

从显示结果可以看到目录下的测试数据：
'/kaggle/input/titanic-survival-data/titanic_data.csv'。

In [ ]:
# Display the folders and files in current directory;
import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

调用pandas的函数 read_csv() 读取 csv 文件内容到DataFrame df_train。

读取数据以后，检查 df_train 的最初几行数据，同时可以看到每一列特征值。有些特征是是数值型的，有一些是字符型的。其中'Survived'是旅客的幸存标志。这一幸存标志在以后的例子中会作为训练数据的标签(the targets for training data)。

In [ ]:
# Load data
df_train = pd.read_csv('/kaggle/input/titanic-survival-data/titanic_data.csv')
# Show first lines of data
df_train.head()

# 数据检查

df_train.shape 显示数据的行数和列数；df_info() 显示数据的基本信息，包括每一列特征值的名称，行数，数值类型，和占用的内存空间。

df_trian.desvribe()显示数据每一列的统计信息：平均值，方差，最大值，最小值，分位数，等。

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

# **缺失数据的处理**

原始数据中往往有数据缺失。以下函数用于检测训练数据和测试数据的缺失率。我们可以看到在数据中，以下的列都有数据缺失：'Cabin', 'Age', 'Embarked', 'Fare'。其中 'Cabin' 的数据缺失率达到77.1%。

In [ ]:
# Function to check the missing percent of a DatFrame;
def check_missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False) * 100 /len(df),2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

In [ ]:
check_missing_data(df_train)

一个简单的处理缺失数据的方式，就是用0和1填补 'Cabin'的缺失数据。因为Cabin本身的字符没有意义，所以在 Cabin缺失的位置填0，并且用1替代Cabin的原有非空字符。

In [ ]:
# Missing data: Cabin has high rate of missing data; insted of deleting the column,
# I will give 1 if Cabin is not null; otherwise 0;
df_train['Cabin']=np.where(df_train['Cabin'].isnull(),0,1)

对于 'Age', 'Fare', 用对应列的平均值填补缺失的数据。而对于 'Embarked'，用相应列中出现最多的字符来代替缺失的值，这通过 dataframe 的函数 data['Embarked'].mode() 来实现。

In [ ]:
# Combine train and test data, fill the missing values;
dataset = [df_train]

# def missing_data(x):
for data in dataset:
    #complete missing age with median
    data['Age'].fillna(data['Age'].mean(), inplace = True)

    #complete missing Embarked with Mode
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)

    #complete missing Fare with median
    data['Fare'].fillna(data['Fare'].mean(), inplace = True)

缺失数据经过处理以后，再次确认df_train已没有缺失数据，并检查 df_train 的最初几行。

In [ ]:
check_missing_data(df_train)

In [ ]:
df_train.head()

# **数据清理**

在之后的联系中，我们将利用训练数据中的信息预测测设数据中的生存状况（即 Survived）。而根据经验，我们相信 'Name', 'Ticket', 'Embarked'三列的信息对生存没有影响。因此将这三列从数据中删除。

In [ ]:
# Delete the irrelavent columns: Name, Ticket (which is ticket code)
drop_column = ['Name','Ticket','Embarked']
df_train.drop(drop_column, axis= 1, inplace = True)

检查一下训练数据中剩下的列：

In [ ]:
df_train.head()

检查冗余信息：相关性是检查冗余信息的有效方法。当某两列的相关系数非常高（接近1.0）的时候，我们认为这两列的信息是冗余的。以下的相关系数矩阵显示这个例子中没有冗余信息。

In [ ]:
df_train.corr()

标准方差用于检测无用信息。当某一列的方差接近0.0时，这一列的所有值几乎时相同的，也就是提供了无用信息。这样的列可以被删除。我们的例子中，方差都远大于0，可以认为没有无用信息。

In [ ]:
df_train.std()

# **非数值变量的处理**

数据中的'Sex'仍然是字符串类型，我们需要将其转化为数值变量。以下的代码中，我们将 'male'用0取代， 'female'用1取代。

In [ ]:
# Convert ‘Sex’ feature into numeric.
genders = {"male": 0, "female": 1}
all_data = [df_train]

for dataset in all_data:
    dataset['Sex'] = dataset['Sex'].map(genders)
df_train['Sex'].value_counts()

数据处理完成后，检查数据。

In [ ]:
df_train.head()

到这一步为止，训练数据已完成检查和预处理。